In [2]:
import ibis

ibis.options.interactive = True

In [3]:
conn = ibis.sqlite.connect('/tmp/geography.db')

In [4]:
countries = conn.table('countries')
countries

,iso_alpha2,iso_alpha3,iso_numeric,fips,name,capital,area_km2,population,continent
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,4975593,AS
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,29121286,AS
3,AG,ATG,28,AC,Antigua and Barbuda,St. Johns,443.0,86754,NA
4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NA
...,...,...,...,...,...,...,...,...,...
247,YE,YEM,887,YM,Yemen,Sanaa,527970.0,23495361,AS
248,YT,MYT,175,MF,Mayotte,Mamoudzou,374.0,159042,AF
249,ZA,ZAF,710,SF,South Africa,Pretoria,1219912.0,49000000,AF
250,ZM,ZMB,894,ZA,Zambia,Lusaka,752614.0,13460305,AF


In [5]:
countries.population.cast('float').sum()

6878963738.0

In [6]:
countries.area_km2.cast('int32').sum()

150012536

In [7]:
expr = (
    countries
    .continent
    .case()
    .when('AF', 'Africa')
    .when('AN', 'Antarctica')
    .when('AS', 'Asia')
    .when('NA', 'North America')
    .when('OC', 'Oceania')
    .when('SA', 'South America')
    .end()
    .name('continent_name'))
expr.value_counts()

,continent_name,count
0,None,54
1,Africa,58
2,Antarctica,5
3,Asia,51
4,North America,42
5,Oceania,28
6,South America,14


In [8]:
expr = (
    countries
    .continent
    .case()
    .when('AF', 'Africa')
    .when('AS', 'Asia')
    .when('NA', 'North America')
    .when('OC', 'Oceania')
    .when('SA', 'South America')
    .end()  # AN goes in the None bin
    .name('continent_name'))
expr.value_counts()

,continent_name,count
0,None,59
1,Africa,58
2,Asia,51
3,North America,42
4,Oceania,28
5,South America,14


In [10]:
expr = (
    ibis.case()
    .when(countries.population > 25_000_000, 'big')
    .when(countries.population < 5_000_00, 'small')
    .else_('medium')
    .end()
    .name('size'))
countries['name', 'population', expr].limit(20)

,name,population,size
0,Andorra,84000,small
1,United Arab Emirates,4975593,medium
2,Afghanistan,29121286,big
3,Antigua and Barbuda,86754,small
4,Anguilla,13254,small
5,Albania,2986952,medium
6,Armenia,2968000,medium
7,Netherlands Antilles,300000,small
8,Angola,13068161,medium
9,Antarctica,0,small


In [11]:
expr = (countries.continent == 'AS').ifelse('Yes', 'No').name('Asia')
countries['name', 'continent', expr].limit(10)

,name,continent,Asia
0,Andorra,EU,No
1,United Arab Emirates,AS,Yes
2,Afghanistan,AS,Yes
3,Antigua and Barbuda,NA,No
4,Anguilla,NA,No
5,Albania,EU,No
6,Armenia,AS,Yes
7,Netherlands Antilles,NA,No
8,Angola,AF,No
9,Antarctica,AN,No
